In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [ ]:
save_time = 15

In [ ]:
left_x = -400
right_x = 600

# chain-chain parameters
m_1 = 1.0
m_2 = 0.5
c_1 = 1.0
c_2 = 1.0
c_12 = 3.0
d_1 = 0.0
d_2 = 0.0
a = 1

# wave packet parameters
omega = 1
beta = 0.03
n_0 = -150
u_0 = 1

# integration parameters
dt = 0.005
t_max = 350

print(f'Input Omega: {omega}.\n',
      f'Min Omega Chain 1: {np.sqrt(d_1/m_1)}.',
      f'Max Omega Chain 1: {np.sqrt((4*c_1+d_1)/m_1)}.\n',
      f'Min Omega Chain 2: {np.sqrt(d_2/m_2)}.',
      f'Max Omega Chain 2: {np.sqrt((4*c_2+d_2)/m_2)}.',
      sep='\n')

In [ ]:
num = np.round(np.arange(left_x, right_x, a)/a)
mass = np.array([m_1 if i < 0 else m_2 for i in num])
foundation_stiffness = np.array([d_1 if j < 0 else d_2 for j in num])
stiffness = np.array([c_1 if k < 0 else c_2 for k in num])
stiffness[np.where(num == -1)[0][0]] = c_12

In [ ]:
def energy(mass, stiffness, foundation_stiffness, vel, disp):
    e = mass/2 * vel**2 + stiffness/4 * (np.roll(disp,-1)-disp)**2+\
        np.roll(stiffness,1)/4 * (np.roll(disp,1)-disp)**2 + foundation_stiffness/2*disp**2
    return e

In [ ]:
def specify_initial_and_boundary(num, beta, n_0, u_0, stiffness, foundation_stiffness, mass, a, omega):

    k_1 = np.arcsin(np.sqrt(mass*(omega**2-foundation_stiffness/mass)/(4*stiffness))) * 2 / a
    g_1 = a/(2*omega)*np.sqrt((omega**2-foundation_stiffness/mass)*((4*stiffness+foundation_stiffness)/mass-omega**2))
    

    disp = u_0 * np.exp(-beta**2/2 * (num - n_0)**2) * np.sin(num * a * k_1)
    vel = -u_0 * np.exp(-beta**2/2 * (num - n_0)**2)
    vel *= (omega * np.cos(k_1*a*num) - beta**2*g_1/a*(num-n_0)*np.sin(num * a * k_1))
    disp[np.where(num>=-1)] = 0
    vel[np.where(num>=-1)] = 0
    
    return disp, vel

In [ ]:
def solver(mass, disp, vel, stiffness, foundation_stiffness, disp_history, vel_history, energy_history, t_max, dt,
           save_time):

    times = np.arange(0, t_max, dt)

    for t in tqdm(times):

        if t % save_time == 0:
            disp_history += [disp]
            vel_history += [vel]
            energy_history += [energy(mass, stiffness, foundation_stiffness, vel, disp)]
        
        # leapfrog synchronized form
        acc1 = (stiffness/mass)*(np.roll(disp,-1)-disp)+(np.roll(stiffness,1)/mass)*(np.roll(disp,1)-disp)-\
            foundation_stiffness/mass*disp
        disp += vel*dt+1/2*acc1*dt**2
        acc2 = (stiffness/mass)*(np.roll(disp,-1)-disp)+(np.roll(stiffness,1)/mass)*(np.roll(disp,1)-disp)-\
            foundation_stiffness/mass*disp
        vel += 1/2*(acc1+acc2)*dt
        
    
    return disp_history, vel_history, energy_history

In [ ]:
disp, vel = specify_initial_and_boundary(num, beta, n_0, u_0, stiffness, foundation_stiffness, mass, a, omega)

plt.plot(num, disp)
plt.title(f'В начальный момент времени')
plt.xlabel('Номер частицы')
plt.ylabel('Перемещение, усл. единиц')
plt.grid()
plt.show()
print(max(disp))

In [ ]:
disp_history = []
vel_history = []
energy_history = []
disp_history, vel_history, energy_history = solver(mass, disp, vel, stiffness, foundation_stiffness, disp_history,
                                                   vel_history, energy_history, t_max, dt, save_time)

plt.plot(num, disp_history[-1])
plt.title(f'В момент времени t={t_max} усл.ед.')
plt.xlabel('Номер частицы')
plt.ylabel('Перемещение, усл. единиц')
plt.grid()
plt.show()

In [ ]:
energy_sum = np.sum(energy_history,axis=1)
energy_left = np.sum(np.array([e[:np.where(num == 0)[0][0]] for e in energy_history]),axis=1)
energy_left /= max(energy_sum)
energy_right = np.sum(np.array([e[np.where(num == 0)[0][0]:len(num)] for e in energy_history]),axis=1)
energy_right /= max(energy_sum)
energy_sum /= max(energy_sum)
saved_times = range(0,t_max,save_time)
plt.plot(saved_times, energy_sum)
plt.title('Полная энергия системы')
plt.xlabel('Время')
plt.ylabel('Энергия')
plt.show()

In [ ]:
history = np.array(energy_history)

In [ ]:
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

fig1, axs = plt.subplots(nrows=2, ncols=1, layout='constrained')

line = axs[0].plot(num, history[0], c='blue', linewidth=1)[0]
line1 = axs[1].plot(saved_times[0], energy_sum[0], c='black', linewidth=1, label="Полная энергия в системе")[0]
line2 = axs[1].plot(saved_times[0], energy_left[0], c='blue', linewidth=1, label="Энергия в левой решётке")[0]
line3 = axs[1].plot(saved_times[0], energy_right[0], c='red', linewidth=1, label="Энергия в правой решётке")[0]

axs[0].grid()
axs[0].set_ylim((min(history.flatten())-0.05,1.2*max(history.flatten())))
axs[0].plot([0,0],[min(history.flatten())-0.05,1.2*max(history.flatten())], c='r', linewidth=0.8)
axs[0].set_xlabel('Номер частицы')
axs[0].set_ylabel('Энергия, усл.ед.')
axs[1].grid()
en_conc = np.concatenate([energy_sum, energy_left, energy_right])
axs[1].set_ylim((0.9*min(en_conc),1.2*max(en_conc)))
axs[1].set_xlim((0,saved_times[-1]))
axs[1].set_title('Энергия в системе')
axs[1].set_xlabel('Время, усл.ед.')
axs[1].set_ylabel('Энергия, усл.ед.')
axs[1].legend(fontsize=7)

def update(frame):
    line.set_xdata(num)
    line.set_ydata(history[frame])
    line1.set_xdata(saved_times[:frame])
    line1.set_ydata(energy_sum[:frame])
    line2.set_xdata(saved_times[:frame])
    line2.set_ydata(energy_left[:frame])
    line3.set_xdata(saved_times[:frame])
    line3.set_ydata(energy_right[:frame])
    axs[0].set_title(f't={save_time*frame}\n(m1={m_1}, m2={m_2}, c1={c_1}, c2={c_2}, c12={c_12}, a={a}, omega={omega})')
    return [line, line1, line2, line3] 

anim = animation.FuncAnimation(fig1, update, frames=len(history))
anim

In [ ]:
#anim.save('1.gif')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(saved_times),y=energy_sum, name='Полная энергия', line=dict(color="#000000")))
fig.add_trace(go.Scatter(x=np.array(saved_times),y=energy_left, name='В левой решётке', line=dict(color="#0433FF")))
fig.add_trace(go.Scatter(x=np.array(saved_times),y=energy_right, name='В правой решётке', line=dict(color="#FF2600")))

fig.update_layout(
    title={'text':"Энергия в системе", 'x':0.5, 'y':0.9},
    xaxis_title="Время, усл.ед.",
    yaxis_title="Энергия, усл.ед.",
    legend_title="Легенда",
    font=dict(
        family="Times New Roman",
        size=18,
        color="black"
    )
)

fig.show()

# Comparison of numerical results with analytical solution

In [ ]:
from sympy import symbols, Abs, I, exp
from sympy.plotting import plot


def transmission_analytical(m_1, m_2, c_1, c_2, d_1, d_2, omega, a):
    c12 = symbols('c12')

    k1 = 2/a*np.arcsin(np.sqrt((m_1*omega**2-d_1)/(4*c_1)))
    k2 = 2/a*np.arcsin(np.sqrt((m_2*omega**2-d_2)/(4*c_2)))

    g1 = a/(2*omega)*np.sqrt((omega**2-d_1/m_1)*((4*c_1+d_1)/m_1-omega**2))
    g2 = a/(2*omega)*np.sqrt((omega**2-d_2/m_2)*((4*c_2+d_2)/m_2-omega**2))

    A_frac = (2*I*c12*np.sin(k1*a))/(c12*(1-exp(-I*k1*a))+c_2*(exp(I*k2*a)-1)*(1+exp(-I*k1*a)*(c12-c_1)/c_1))
    trans_coeff = m_2*g2/(m_1*g1)*(Abs(A_frac))**2

    return trans_coeff


def transmission_numerical(m_1, m_2, c_1, c_2, d_1, d_2, omega, a, c_12):

    #left_x = -400
    #right_x = 600
    #beta = 0.03
    #n_0 = -150
    #u_0 = 1

    num = np.round(np.arange(left_x, right_x, a)/a)
    mass = np.array([m_1 if i < 0 else m_2 for i in num])
    foundation_stiffness = np.array([d_1 if j < 0 else d_2 for j in num])
    stiffness = np.array([c_1 if k < 0 else c_2 for k in num])
    stiffness[np.where(num == -1)[0][0]] = c_12

    disp, vel = specify_initial_and_boundary(num, beta, n_0, u_0, stiffness, foundation_stiffness, mass, a, omega)
    
    dt = 0.01
    #t_max = 350
    
    disp_history = []
    vel_history = []
    energy_history = []
    disp_history, vel_history, energy_history = solver(mass, disp, vel, stiffness, foundation_stiffness, disp_history,
                                                       vel_history, energy_history, t_max, dt, save_time)
    
    energy_sum = np.sum(energy_history,axis=1)
    energy_right = np.sum(np.array([e[np.where(num == 0)[0][0]:len(num)] for e in energy_history]),axis=1)

    #print(max(energy_right/energy_sum))

    trans_coeff = max(energy_right/energy_sum)

    return trans_coeff

In [ ]:
trans_a = transmission_analytical(m_1, m_2, c_1, c_2, d_1, d_2, omega, a)
trans_num = lambda x: transmission_numerical(m_1, m_2, c_1, c_2, d_1, d_2, omega, a, x)


c12_values = np.concatenate([np.linspace(0,1,10), np.linspace(1.3,5.3,10)])
trans_a_values = [trans_a.evalf(subs={'c12':i}) for i in c12_values]
trans_num_values = [trans_num(j) for j in c12_values]

In [ ]:
plt.close(fig1)
plt.rcParams["animation.html"] = "none"
plt.figure()
plt.plot(c12_values, trans_num_values, c='blue', label='Численно')
plt.plot(c12_values, trans_a_values, '--', c='red', label='Аналитически')
plt.title('Interface stiffness dependence of the transmission coefficient\n'+\
          f'(m1={m_1}, m2={m_2}, c1={c_1}, c2={c_2}, a={a}, omega={omega})')
plt.xlabel('c12')
plt.ylabel('T')
plt.grid()
plt.legend()
plt.show()